In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [2]:
from datasets import load_dataset

from seq import Widget

ds = load_dataset("ajaykarthick/imdb-movie-reviews")["test"]

True


In [3]:
w = Widget(sequences=[m["review"] for m in ds if m["label"] == 1][:1000])
w

Widget(feature_ids=[3, 213, 29, 66, 10, 5, 162, 58, 151, 59], ids=[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 10, 6, 11…

In [93]:
import numpy as np

ids = np.array(widget.ids)
feature_ids = np.array(widget.feature_ids)

ids.shape, feature_ids.shape

NameError: name 'widget' is not defined

In [94]:
# id that feature_ids contains

print(ids.flatten().shape)
np.isin(ids.flatten(), feature_ids.flatten()).sum()

NameError: name 'ids' is not defined

In [111]:
import numpy as np
from numpy.random import choice

n_row = 10000
n_label = 10
n_window = 64
A = choice(
  n_label + 1, (n_row, n_window), p=[0.92] + [0.08 / n_label] * n_label
)
A = A[A.sum(axis=1) > 0]
A.shape

(9957, 64)

In [112]:
A_expanded = A[:, np.newaxis, :]
dist_matrix = (
  (A_expanded == A_expanded.transpose(1, 0, 2)) * (A_expanded != 0)
).sum(axis=2)
np.fill_diagonal(dist_matrix, 0)
dist_matrix.shape

(9957, 9957)

In [113]:
import networkx as nx

G = nx.Graph()
for i in range(len(dist_matrix)):
  for j in range(i + 1, len(dist_matrix)):
    G.add_edge(i, j, weight=n_window - dist_matrix[i, j])


answer = nx.algorithms.approximation.christofides(G)

KeyboardInterrupt: 

In [109]:
def get_scores(matrix):
  return (((matrix[:-1, :] - matrix[1:, :]) == 0) * (matrix[:-1, :] != 0)).sum()


print(get_scores(A))
print(get_scores(A[np.array(answer)]))

37
983
